In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch

# Force PyTorch to reinitialize CUDA
torch.cuda.device(0)  # Ensure we are on the correct GPU
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_peak_memory_stats()
torch.cuda.empty_cache()

# Force exit from previous CUDA errors
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

c:\Users\rewin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\cuda\memory.py:391: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [2]:
df = pd.read_csv("C:\\Users\\rewin\\Desktop\\test,train\\train.csv")
#df.columns = ['sentiment', 'text']
df.head()

,sentiment,text
0,positive,@JetBlue @JayVig I like the inflight snacks! I...
1,positive,@VirginAmerica thanks guys! Sweet route over t...
2,negative,@USAirways Your exchange/credit policies are w...
3,negative,@USAirways but in the meantime I'll be sleepin...
4,negative,@VirginAmerica hold times at call center are a...


In [3]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Enables CUDA debugging
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from evaluate import load  # NEW: `evaluate.load()` replaces `load_metric`
from sklearn.metrics import accuracy_score

from evaluate import load
metric = load("accuracy")


In [4]:
MODEL_NAME = "microsoft/deberta-v3-large"
MAX_LENGTH = 128

import pandas as pd

#   Load train.csv and test.csv
train_df = pd.read_csv(r"C:\Users\rewin\Desktop\test,train\train.csv")
test_df = pd.read_csv(r"C:\Users\rewin\Desktop\test,train\test.csv")

#   Rename columns to match in both files
train_df = train_df.rename(columns={"sentiment": "sentiment"})  # Rename "sentiment" to "sentiment" for training
test_df = test_df.rename(columns={"id": "sentiment"})  # Rename "id" to "sentiment" (will be removed from test later)

#   Save updated files
train_df.to_csv(r"C:\Users\rewin\Desktop\test,train\train.csv", index=False)
test_df.to_csv(r"C:\Users\rewin\Desktop\test,train\test.csv", index=False)

from datasets import load_dataset

#   Load dataset with fixed column names
dataset = load_dataset(
    "csv",
    data_files={
        "train": r"C:\Users\rewin\Desktop\test,train\train.csv",
        "test": r"C:\Users\rewin\Desktop\test,train\test.csv"
    }
)

#   Remove the 'sentiment' column only from the test split (we don't have true sentiments there)
dataset["test"] = dataset["test"].remove_columns(["sentiment"])

#   Convert training sentiments to integers, mapping negative=0, neutral=1, positive=2
def convert_sentiment(example):
    lbl = str(example["sentiment"]).strip().lower()

    # Convert text sentiments to numeric ones
    if lbl in ["positive", "pos"]:
        example["sentiment"] = 2
    elif lbl in ["negative", "neg"]:
        example["sentiment"] = 0
    elif lbl in ["neutral"]:
        example["sentiment"] = 1
    else:
        try:
            num = int(lbl)
            if num in [0, 1, 2]:  # Only allow valid values
                example["sentiment"] = num
            else:
                print(f"❌ Out-of-bound sentiment found: {lbl} - Setting to 1 (neutral)")
                example["sentiment"] = 1
        except:
            print(f"❌ Unexpected sentiment: {lbl} - Setting to 1 (neutral)")
            example["sentiment"] = 1

    return example

# Step 2: Ensure sentiments are correctly mapped
dataset["train"] = dataset["train"].map(convert_sentiment)

# Step 3: Convert sentiments to int64 (long tensors for PyTorch)
import torch
dataset["train"] = dataset["train"].map(lambda x: {"sentiment": int(x["sentiment"])})

# Confirm sentiment types are correct
print("Label data type:", type(dataset["train"][0]["sentiment"]))  # Should print: <class 'int'>




# Print dataset structure (for verification)
print("Train Features:", dataset["train"].features)
print("Test Features:", dataset["test"].features)
print(f"Training Samples: {len(dataset['train'])}")
print(f"Testing Samples: {len(dataset['test'])}")

from transformers import AutoTokenizer

# Load RoBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

#   Define tokenization function
def tokenize_function(example):
    tokenized = tokenizer(example["text"], padding="max_length", truncation=True)
    # If the sentiment exists in the example, pass it through
    if "sentiment" in example:
        tokenized["sentiment"] = example["sentiment"]
    return tokenized


#   Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

#   Print tokenization check (Only 1 sample)
print("Example Tokenized Text:", tokenized_datasets["train"][0])



Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Label data type: <class 'int'>
Train Features: {'sentiment': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None)}
Test Features: {'text': Value(dtype='string', id=None)}
Training Samples: 10000
Testing Samples: 3000


c:\Users\rewin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Example Tokenized Text: {'sentiment': 2, 'text': "@JetBlue @JayVig I like the inflight snacks! I'm flying with you guys on 2/28! #JVMChat", 'input_ids': [1, 1944, 31333, 16010, 1944, 39094, 1989, 8843, 273, 334, 262, 267, 18051, 8262, 300, 273, 280, 358, 4021, 275, 274, 1860, 277, 392, 320, 3686, 300, 953, 2252, 23334, 31923, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [5]:
classes = df.sentiment.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
class_idx2tok = dict((k, v) for k, v in enumerate(classes))
print(class_tok2idx)
print(class_idx2tok)

{'positive': 0, 'negative': 1, 'neutral': 2}
{0: 'positive', 1: 'negative', 2: 'neutral'}


In [6]:
df['sentiment'] = df['sentiment'].apply(lambda x: class_tok2idx[x])
df.head()

,sentiment,text
0,0,@JetBlue @JayVig I like the inflight snacks! I...
1,0,@VirginAmerica thanks guys! Sweet route over t...
2,1,@USAirways Your exchange/credit policies are w...
3,1,@USAirways but in the meantime I'll be sleepin...
4,1,@VirginAmerica hold times at call center are a...


In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large", num_labels=2)

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

# Tokenize sequence
sequence_0 = "@united I will never fly with you again. Period."
seq0_tokens = tokenizer(sequence_0, return_tensors="pt")

# Print tokenized input
print(f"Number of tokens in seq0 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)

# Get model predictions
logits = model(**seq0_tokens).logits
print("Logits:", logits)



Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\rewin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Number of tokens in seq0 is 14
{'input_ids': tensor([[    1,  1944, 57116,   273,   296,   518,  3247,   275,   274,   541,
           260, 12914,   260,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Logits: tensor([[0.0651, 0.0145]], grad_fn=<AddmmBackward0>)


In [8]:
sequence_1 = "@united Delayed flight, but you did the best you could. Thank you united crew."
seq1_tokens = tokenizer(sequence_1, return_tensors="pt")
print(f"number of tokens in seq1 is {len(seq1_tokens['input_ids'].flatten())}")
model(**seq1_tokens).logits

number of tokens in seq1 is 20


tensor([[0.0602, 0.0082]], grad_fn=<AddmmBackward0>)

In [9]:
sequence_2 = "@united There is a reason why a only fly united, you're the best!"
seq2_tokens = tokenizer(sequence_2, return_tensors="pt")
print(f"number of tokens in seq2 is {len(seq2_tokens['input_ids'].flatten())}")
model(**seq2_tokens).logits

number of tokens in seq2 is 20


tensor([[0.0450, 0.0470]], grad_fn=<AddmmBackward0>)

In [10]:
print(dataset["train"][0])


{'sentiment': 2, 'text': "@JetBlue @JayVig I like the inflight snacks! I'm flying with you guys on 2/28! #JVMChat"}


In [11]:
from datasets import Dataset

# Convert Pandas DataFrame to Hugging Face Dataset
ds_raw = Dataset.from_pandas(df[['sentiment','text']])

# Print first row to verify
print(ds_raw[0])


{'sentiment': 0, 'text': "@JetBlue @JayVig I like the inflight snacks! I'm flying with you guys on 2/28! #JVMChat"}


In [12]:
from datasets import load_dataset

raw_datasets = load_dataset(
    "csv",
    data_files={
        "train": r"C:\Users\rewin\Desktop\test,train\train.csv",
        "test": r"C:\Users\rewin\Desktop\test,train\test.csv"
    }
)



def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)

# Only remove columns you don't need. For example, if you only want to remove "text" (after tokenizing it)
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=[col for col in raw_datasets["train"].column_names if col != "sentiment"]
)



ds = ds_raw.map(tokenize_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [13]:
ds[0]

{'sentiment': 0,
 'text': "@JetBlue @JayVig I like the inflight snacks! I'm flying with you guys on 2/28! #JVMChat",
 'input_ids': [1,
  1944,
  31333,
  16010,
  1944,
  39094,
  1989,
  8843,
  273,
  334,
  262,
  267,
  18051,
  8262,
  300,
  273,
  280,
  358,
  4021,
  275,
  274,
  1860,
  277,
  392,
  320,
  3686,
  300,
  953,
  2252,
  23334,
  31923,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [14]:
ds = ds.shuffle(seed=42)
ds[0]

{'sentiment': 1,
 'text': '@AmericanAir 11 out of 11 delayed flights, you suck and getting worse',
 'input_ids': [1,
  1944,
  5466,
  17127,
  762,
  321,
  265,
  762,
  7895,
  5181,
  261,
  274,
  37535,
  263,
  646,
  2687,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [15]:
train_prop = 0.70
ds_train = ds.select(range(int(len(ds)*train_prop)))
ds_eval = ds.select(range(int(len(ds)*train_prop), len(ds)))

In [16]:
print(f"len(ds_train) = {len(ds_train)}")
print(f"len(ds_eval) = {len(ds_eval)}")

len(ds_train) = 7000
len(ds_eval) = 3000


In [17]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [18]:
print(tokenized_datasets["train"].features)


{'sentiment': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


In [19]:
# Check unique sentiments in the training split
unique_sentiments = set(dataset["train"]["sentiment"])
print("Unique training sentiments:", unique_sentiments)


Unique training sentiments: {0, 1, 2}


In [20]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))


CUDA available: True
Current device: 0
Device name: NVIDIA GeForce RTX 4070 Ti SUPER


In [21]:
from torch.utils.data import DataLoader

# Create a DataLoader manually to check sentiments
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=4, shuffle=True)

# Print first batch
for batch in train_dataloader:
    print("Batch sentiments:", batch["sentiment"])  # Ensure this prints only values in {0, 1, 2}
    break


Batch sentiments: ['positive', 'negative', 'negative', 'neutral']


In [22]:
print("Model's num_labels:", model.config.num_labels)


Model's num_labels: 2


In [23]:
print("First tokenized sample:", tokenized_datasets["train"][0])


First tokenized sample: {'sentiment': 'positive', 'input_ids': [1, 1944, 31333, 16010, 1944, 39094, 1989, 8843, 273, 334, 262, 267, 18051, 8262, 300, 273, 280, 358, 4021, 275, 274, 1860, 277, 392, 320, 3686, 300, 953, 2252, 23334, 31923, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [24]:
import torch
torch.cuda.empty_cache()


In [25]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0))
print("GPU Memory Allocated:", torch.cuda.memory_allocated() / 1024**3, "GB")
print("GPU Memory Reserved:", torch.cuda.memory_reserved() / 1024**3, "GB")


CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4070 Ti SUPER
GPU Memory Allocated: 0.0 GB
GPU Memory Reserved: 0.0 GB


In [26]:
import numpy as np
import evaluate
import torch
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoTokenizer
from torch.optim import AdamW
from transformers import EarlyStoppingCallback

# Load accuracy metric
accuracy = evaluate.load("accuracy")

# Define a function to compute accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    if len(labels) == 0:
        return {}
    return {"eval_accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"]}

# Load model and tokenizer (using 3 labels in this example)
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")
model.config.hidden_dropout_prob = 0.4
model.config.attention_probs_dropout_prob = 0.4
model.config.sentiment_smoothing_factor = 0.1

# Define the label mapping for your three classes
# Define the label mapping for string sentiments
label_map = {"negative": 0, "neutral": 1, "positive": 2}

def add_labels(example):
    sentiment_val = example["sentiment"]
    # If the sentiment value is a string and is a digit, convert it to int
    if isinstance(sentiment_val, str):
        if sentiment_val.isdigit():
            # Assume it's already the numeric label as string
            example["labels"] = int(sentiment_val)
        elif sentiment_val in label_map:
            example["labels"] = label_map[sentiment_val]
        else:
            raise KeyError(f"Unexpected sentiment value: {sentiment_val}")
    else:
        # If it's not a string, assume it's already numeric
        example["labels"] = int(sentiment_val)
    return example

# Apply the mapping to your tokenized dataset
tokenized_datasets = tokenized_datasets.map(add_labels)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])



#print(tokenized_datasets["train"][0]) # Should show keys: "input_ids", "attention_mask", "labels", etc.


# Split the dataset for training and evaluation
train_size = int(0.70 * len(tokenized_datasets["train"]))
val_size = len(tokenized_datasets["train"]) - train_size

train_dataset = tokenized_datasets["train"].select(range(train_size))
eval_dataset = tokenized_datasets["train"].select(range(train_size, train_size + val_size))

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-6, weight_decay=0.02)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
    save_total_limit=1,  # Keeps only the best model
    adam_epsilon=2.04e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=4,
    warmup_steps=100,
    weight_decay=0.02,
    learning_rate=6.05e-6,
    lr_scheduler_type="cosine",
    logging_steps=10,
    eval_steps=500,
    logging_first_step=True,
    load_best_model_at_end=True,
    report_to="none",
    fp16_full_eval=True,  # Ensures evaluation runs in mixed precision (faster inference)

)







# Debug: Print a sample from the training and evaluation datasets
#print("🔍 Train dataset sample:", train_dataset[0])
#print("🔍 Eval dataset sample:", eval_dataset[0])

# Ensure that the eval dataset has labels
if "labels" not in eval_dataset.features:
    raise ValueError("❌ `eval_dataset` is missing 'labels' field. Check your preprocessing step.")



# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    compute_metrics=compute_metrics
 )
# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()

# If eval_accuracy is missing, compute it manually:
if "eval_accuracy" not in results or results["eval_accuracy"] is None:
    print("⚠️ `eval_accuracy` not found in Trainer output. Computing manually...")
    eval_dataloader = trainer.get_eval_dataloader()
    all_logits, all_labels = [], []
    for batch in eval_dataloader:
        batch = {k: v.cuda() for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
            logits = outputs.logits.cpu().numpy()
            labels = batch["labels"].cpu().numpy()
            all_logits.append(logits)
            all_labels.append(labels)
    all_logits = np.concatenate(all_logits, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    eval_accuracy = compute_metrics((all_logits, all_labels))
    results.update(eval_accuracy)

print(results)





Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\rewin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

c:\Users\rewin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.531700,0.548494,0.761000


SafetensorError: Error while serializing: IoError(Os { code: 112, kind: StorageFull, message: "There is not enough space on the disk." })

In [ ]:
df_test = pd.read_csv("C:\\Users\\rewin\\Desktop\\test,train\\test.csv")
df_test.head()

,sentiment,text
0,0,@JetBlue okay awesome! Thank you!
1,1,@SouthwestAir wanting a ⭐️ for the ✈️ finding ...
2,2,"@united Funny thing happened, we arrived on ou..."
3,3,@united 2nd Late Flight flight today
4,4,@USAirways your service at PHL is abysmal. An ...


In [ ]:
ds_test_raw = Dataset.from_pandas(df_test)
ds_test_raw[0]

{'sentiment': 0, 'text': '@JetBlue okay awesome! Thank you!'}

In [ ]:
ds_test = ds_test_raw.map(tokenize_function, batched=True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
preds = trainer.predict(test_dataset=ds_test)

In [ ]:
test_preds = np.apply_along_axis(np.argmax, 1, preds.predictions) 
#ds_test['preds'] = pd.Series(test_preds.tolist())

In [ ]:
df_test['preds'] = test_preds.tolist()
df_test['sentiment'] = df_test['preds'].apply(lambda x: class_idx2tok[x])
df_test.head()

,sentiment,text,preds
0,neutral,@JetBlue okay awesome! Thank you!,2
1,positive,@SouthwestAir wanting a ⭐️ for the ✈️ finding ...,0
2,positive,"@united Funny thing happened, we arrived on ou...",0
3,positive,@united 2nd Late Flight flight today,0
4,positive,@USAirways your service at PHL is abysmal. An ...,0


In [ ]:
print(df_test.columns)


Index(['sentiment', 'text', 'preds'], dtype='object')


In [ ]:
df_submission = df_test[['preds']].copy()  # Get only predictions
df_submission.rename(columns={'preds': 'sentiment'}, inplace=True)  # Rename 'preds' to 'sentiment'

# Convert numeric predictions back to sentiment labels
sentiment_map = {0: "negative", 1: "neutral", 2: "positive"}
df_submission["sentiment"] = df_submission["sentiment"].map(sentiment_map)

# Create a correct ID column with sequential numbers (0, 1, 2, 3, ...)
df_submission.insert(0, 'id', range(len(df_submission)))  

# Save to CSV
df_submission.to_csv("submission.csv", index=False)

print(df_submission.head())  


   id sentiment
0   0  positive
1   1  negative
2   2  negative
3   3  negative
4   4  negative


In [ ]:
df_submission.to_csv('submission.csv', index=False)
print(os.getcwd()) 


c:\Users\rewin\Downloads
